### Home work: build a regression model for the conversion rate

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often,more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed Attribute Information:

Input variables:
bank client data:
- age (numeric)
- job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
- marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
- education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
- default: has credit in default? (categorical: 'no','yes','unknown')
- housing: has housing loan? (categorical: 'no','yes','unknown')
- loan: has personal loan? (categorical: 'no','yes','unknown')
related with the last contact of the current campaign:
- contact: contact communication type (categorical: 'cellular','telephone') 
- month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
- day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
- duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
 other attributes:
- campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
- pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
- previous: number of contacts performed before this campaign and for this client (numeric)
- poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
social and economic context attributes
- emp.var.rate: employment variation rate - quarterly indicator (numeric)
- cons.price.idx: consumer price index - monthly indicator (numeric) 
- cons.conf.idx: consumer confidence index - monthly indicator (numeric) 
- euribor3m: euribor 3 month rate - daily indicator (numeric)
- nr.employed: number of employees - quarterly indicator (numeric)

y - has the client subscribed a term deposit? (binary: 'yes','no')

In [115]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [116]:
df=pd.read_csv('bank-additional-full.csv', sep=';')

In [117]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [118]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


Get all continuous variables

In [119]:
continuous_vars = list(df.describe().columns)
continuous_vars

['age',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed']

Create target variable (the EDA was already completed in Project One)

In [120]:
df['conversion'] = df['y'].apply(lambda x:1 if x == 'yes' else 0)

In [121]:
categorical_vars = [x for x in list(df.columns) if x not in continuous_vars and x not in ['y','conversion']]
categorical_vars

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'poutcome']

Categorical variable encoding (let's only use 'marital' and 'education')

In [122]:
columns_to_encode=['marital', 'default']

categorical_features = []
for col in columns_to_encode:
    encoded_df = pd.get_dummies(df[col])
    encoded_df.columns = [col.replace(' ', '.') + '.' + x for x in encoded_df.columns]
    
    categorical_features += list(encoded_df.columns)
    
    df = pd.concat([df, encoded_df], axis=1)

In [123]:
all_features = continuous_vars + categorical_features
print(all_features)

['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'marital.divorced', 'marital.married', 'marital.single', 'marital.unknown', 'default.no', 'default.unknown', 'default.yes']


In [124]:
df[all_features]

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,marital.divorced,marital.married,marital.single,marital.unknown,default.no,default.unknown,default.yes
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,1,0,0,1,0,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,1,0,0,0,1,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,1,0,0,1,0,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,1,0,0,1,0,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,1,0,0,1,0,0
5,45,198,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,1,0,0,0,1,0
6,59,139,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,1,0,0,1,0,0
7,41,217,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,1,0,0,0,1,0
8,24,380,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,1,0,1,0,0
9,25,50,1,999,0,1.1,93.994,-36.4,4.857,5191.0,0,0,1,0,1,0,0


Fit a new logistic regression model

In [125]:
import statsmodels.formula.api as sm

logit = sm.Logit(
     df['conversion'], 
    df[all_features])

In [126]:
logit_fit = logit.fit()

         Current function value: 0.218774
         Iterations: 35


C:\Users\kangd\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [127]:
logit_fit.summary()

C:\Users\kangd\Anaconda3\lib\site-packages\statsmodels\base\model.py:1092: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
C:\Users\kangd\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\kangd\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\kangd\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             conversion   No. Observations:                41188
Model:                          Logit   Df Residuals:                    41172
Method:                           MLE   Df Model:                           15
Date:                Fri, 02 Aug 2019   Pseudo R-squ.:                  0.3785
Time:                        22:36:17   Log-Likelihood:                -9010.8
converged:                      False   LL-Null:                       -14499.
                                        LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
age                  0.0078      0.002      4.190      0.000       0.004       0.011
duration             0.0046    7.2e-05     63.478      0.000       0.004       0.005
campaign            -0.0419      0.011     -3.682      0.000      -0.064      -0.020
pdays               -0.0019   8.45e-05    -22.135      0.000      -0.002      -0.002
previous            -0.2612      0.038     -6.886      0.000      -0.336      -0.187
emp.var.rate        -0.6146      0.067     -9.117      0.000      -0.747      -0.483
cons.price.idx       0.6184      0.100      6.201      0.000       0.423       0.814
cons.conf.idx        0.0297      0.006      5.061      0.000       0.018       0.041
euribor3m            0.0703      0.085      0.831      0.406      -0.095       0.236
nr.employed         -0.0065      0.001     -4.467      0.000      -0.009      -0.004
marital.divorced   -11.9018        nan        nan        nan         nan         nan
marital.married    -11.9299        nan        nan        nan         nan         nan
marital.single     -11.6905        nan        nan        nan         nan         nan
marital.unknown    -11.8363        nan        nan        nan         nan         nan
default.no         -13.9466        nan        nan        nan         nan         nan
default.unknown    -14.4560        nan        nan        nan         nan         nan
default.yes        -18.9561        nan        nan        nan         nan         nan
====================================================================================
"""